<a href="https://www.kaggle.com/tunguz/tps-dec-2021-simple-linear-baseline-with-fe?scriptVersionId=82609688" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from cuml.linear_model import LogisticRegression
from cuml.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import RobustScaler
import gc



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-dec-2021/train.csv
/kaggle/input/tabular-playground-series-dec-2021/test.csv


In [2]:
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/sample_submission.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/train.csv')
test = pd.read_csv('/kaggle/input/tabular-playground-series-dec-2021/test.csv')

In [3]:
train.drop(columns=['Soil_Type7', 'Soil_Type15', 'Soil_Type1', 'Id'], inplace=True) 
test.drop(columns=['Soil_Type7', 'Soil_Type15', 'Soil_Type1', 'Id'], inplace=True) 

In [4]:
new_names = {
    "Horizontal_Distance_To_Hydrology": "x_dist_hydrlgy",
    "Vertical_Distance_To_Hydrology": "y_dist_hydrlgy",
    "Horizontal_Distance_To_Roadways": "x_dist_rdwys",
    "Horizontal_Distance_To_Fire_Points": "x_dist_firepts"
}

train.rename(new_names, axis=1, inplace=True)
test.rename(new_names, axis=1, inplace=True)

train["Aspect"][train["Aspect"] < 0] += 360
train["Aspect"][train["Aspect"] > 359] -= 360

test["Aspect"][test["Aspect"] < 0] += 360
test["Aspect"][test["Aspect"] > 359] -= 360

# Manhhattan distance to Hydrology
train["mnhttn_dist_hydrlgy"] = np.abs(train["x_dist_hydrlgy"]) + np.abs(train["y_dist_hydrlgy"])
test["mnhttn_dist_hydrlgy"] = np.abs(test["x_dist_hydrlgy"]) + np.abs(test["y_dist_hydrlgy"])

# Euclidean distance to Hydrology
train["ecldn_dist_hydrlgy"] = (train["x_dist_hydrlgy"]**2 + train["y_dist_hydrlgy"]**2)**0.5
test["ecldn_dist_hydrlgy"] = (test["x_dist_hydrlgy"]**2 + test["y_dist_hydrlgy"]**2)**0.5

train.loc[train["Hillshade_9am"] < 0, "Hillshade_9am"] = 0
test.loc[test["Hillshade_9am"] < 0, "Hillshade_9am"] = 0

train.loc[train["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0
test.loc[test["Hillshade_Noon"] < 0, "Hillshade_Noon"] = 0

train.loc[train["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0
test.loc[test["Hillshade_3pm"] < 0, "Hillshade_3pm"] = 0

train.loc[train["Hillshade_9am"] > 255, "Hillshade_9am"] = 255
test.loc[test["Hillshade_9am"] > 255, "Hillshade_9am"] = 255

train.loc[train["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255
test.loc[test["Hillshade_Noon"] > 255, "Hillshade_Noon"] = 255

train.loc[train["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255
test.loc[test["Hillshade_3pm"] > 255, "Hillshade_3pm"] = 255

features_Hillshade = ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm']
soil_features = [x for x in train.columns if x.startswith("Soil_Type")]
wilderness_features = [x for x in train.columns if x.startswith("Wilderness_Area")]

def addFeature(X):
    # Thanks @mpwolke : https://www.kaggle.com/mpwolke/tooezy-where-are-you-no-camping-here
    X["Soil_Count"] = X[soil_features].apply(sum, axis=1)

    # Thanks @yannbarthelemy : https://www.kaggle.com/yannbarthelemy/tps-december-first-simple-feature-engineering
    X["Wilderness_Area_Count"] = X[wilderness_features].apply(sum, axis=1)
    X["Hillshade_mean"] = X[features_Hillshade].mean(axis=1)
    X['amp_Hillshade'] = X[features_Hillshade].max(axis=1) - X[features_Hillshade].min(axis=1)
    
addFeature(train)
addFeature(test)

cols = [
    "Elevation",
    "Aspect",
    "mnhttn_dist_hydrlgy",
    "ecldn_dist_hydrlgy",
    "Slope",
    "x_dist_hydrlgy",
    "y_dist_hydrlgy",
    "x_dist_rdwys",
    "Hillshade_9am",
    "Hillshade_Noon",
    "Hillshade_3pm",
    "x_dist_firepts",
    
    "Soil_Count","Wilderness_Area_Count","Hillshade_mean","amp_Hillshade"
]

scaler = RobustScaler()
train[cols] = scaler.fit_transform(train[cols])
test[cols] = scaler.transform(test[cols])

features = test.columns

perm_dict = {1:2, 2:1, 3:3, 4:6, 7:4, 6:5, 5:7,}

train['Cover_Type'].replace(perm_dict, inplace=True)

train = train[train.Cover_Type !=7]

def reduce_mem_usage(df, verbose=True):
    numerics = ['int8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
 
    return df

train = reduce_mem_usage(train)
test = reduce_mem_usage(test)

Mem. usage decreased to 434.88 Mb (75.8% reduction)
Mem. usage decreased to 100.14 Mb (77.0% reduction)


In [5]:
target = train[['Cover_Type']].values
train.drop(['Cover_Type'], axis=1, inplace=True)

In [6]:
inv_perm = {v: k for k, v in perm_dict.items()}
inv_perm

{2: 1, 1: 2, 3: 3, 6: 4, 4: 7, 5: 6, 7: 5}

In [7]:
train_oof = np.zeros((train.shape[0],))
test_preds = 0
train_oof.shape

(3999999,)

In [8]:
%%time
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=7, shuffle=True)

for jj, (train_index, val_index) in enumerate(kf.split(train)):
    print("Fitting fold", jj+1)
    train_features = train.values[train_index,:]
    train_target = target[train_index]

    
    val_features = train.values[val_index,:]
    val_target = target[val_index]
    print(np.unique(train_target, return_counts=True))
    print(np.unique(val_target, return_counts=True))

    
    model = LogisticRegression(C=1, solver='qn', max_iter=200, tol=0.000000001)
    model.fit(train_features, train_target)
    val_pred = model.predict(val_features)
    train_oof[val_index] = val_pred.flatten()
    test_preds += model.predict_proba(test.values)/n_splits
    print(accuracy_score(val_target,val_pred))
    del model
    gc.collect()
    gc.collect()

Fitting fold 1
(array([1, 2, 3, 4, 5, 6], dtype=int8), array([1809273, 1174631,  156645,   49947,    9190,     313]))
(array([1, 2, 3, 4, 5, 6], dtype=int8), array([452814, 293505,  39067,  12314,   2236,     64]))
[W] [17:07:16.154296] L-BFGS stopped, because the line search failed to advance (step delta = 0.000000)
0.9174712300300598
Fitting fold 2
(array([1, 2, 3, 4, 5, 6], dtype=int8), array([1809862, 1174467,  156700,   49585,    9081,     304]))
(array([1, 2, 3, 4, 5, 6], dtype=int8), array([452225, 293669,  39012,  12676,   2345,     73]))
[W] [17:07:29.702547] L-BFGS stopped, because the line search failed to advance (step delta = 0.000000)
0.9175437688827515
Fitting fold 3
(array([1, 2, 3, 4, 5, 6], dtype=int8), array([1809638, 1174622,  156524,   49758,    9174,     283]))
(array([1, 2, 3, 4, 5, 6], dtype=int8), array([452449, 293514,  39188,  12503,   2252,     94]))
[W] [17:07:43.105679] L-BFGS stopped, because the line search failed to advance (step delta = 0.000000)
0.917

In [9]:
print(accuracy_score(target,train_oof))

0.9174404740333557


In [10]:
test_preds

array([[4.83984081e-07, 9.98542786e-01, 7.64902739e-04, 4.39791213e-04,
        1.28073143e-08, 2.52032274e-04],
       [2.01009470e-06, 9.89724755e-01, 1.02712652e-02, 4.11759174e-07,
        4.36197837e-07, 1.08640290e-06],
       [3.62904620e-06, 9.16071415e-01, 8.39079097e-02, 1.22049695e-08,
        1.69222531e-05, 1.14745674e-07],
       ...,
       [4.45719877e-07, 9.99852717e-01, 1.46700404e-04, 1.29427380e-09,
        1.19733095e-08, 4.86754637e-08],
       [6.73594514e-11, 1.05780261e-02, 9.84993458e-01, 1.65206644e-14,
        4.42853291e-03, 6.09783762e-14],
       [2.35316975e-07, 8.94741714e-02, 8.70794494e-08, 9.02931213e-01,
        2.53286780e-12, 7.59440940e-03]], dtype=float32)

In [11]:
preds = np.argmax(test_preds, axis=1)
preds

array([1, 1, 1, ..., 1, 2, 3])

In [12]:
np.unique(preds, return_counts=True)

(array([1, 2, 3, 4, 5]), array([501793, 402754,  83988,   8986,   2479]))

In [13]:
preds[np.where(preds == 0)[0]] = 1

In [14]:
sample_submission['Cover_Type'] = preds
sample_submission['Cover_Type'].replace(inv_perm, inplace=True)
sample_submission.head()

,Id,Cover_Type
0,4000000,2
1,4000001,2
2,4000002,2
3,4000003,2
4,4000004,2


In [15]:
np.unique(sample_submission['Cover_Type'].values)

array([1, 2, 3, 6, 7])

In [16]:
sample_submission.to_csv('submission.csv', index=False)

